In [0]:
import pandas as pd
#1~9호선 전체역별 모델을 제작하고자 하였으나 지하철 승하차 데이터에 전체역이 나와있지 않아 2019년 5월 기준 가장 많은 승하차 인원 top4와
#추가적으로 서울의 대표적 나들이 장소와 가까운 역 4군데를 선정하였다. 이태원 같은 경우는 코로나 확진으로 많은 피해를 받는 구이기 때문에
#포함시켜보았다.
import seaborn as sns
import matplotlib
import matplotlib.dates as md
import matplotlib.pyplot as plt
from datetime import datetime
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib 
from sklearn.metrics import make_scorer

# 시계열
from fbprophet import Prophet
from datetime import datetime as dt
from statsmodels.tsa.arima_model import ARIMA
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
	
mpl.rcParams['axes.unicode_minus'] = False
# 그래프에서 마이너스 폰트 깨질 경우 대비
	
path = '/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf'
font_name = fm.FontProperties(fname=path, size=18).get_name()
plt.rc('font', family=font_name)
fm._rebuild() # 이걸 해줘야 plt.rc가 작동

plt.plot(np.random.randn(4, 8), np.random.randn(4,8), 'bo--')
plt.title('타이틀')
plt.xlabel('X 라벨')
plt.ylabel('Y 라벨')
plt.show()
	

FileNotFoundError: ignored

In [43]:
def mae(prediction, correct):
    prediction = np.array(prediction)
    correct = np.array(correct)
    
    difference = correct - prediction
    abs_val = abs(difference)
    
    score = abs_val.mean()
    
    return score

mae_scorer = make_scorer(mae)
mae_scorer

make_scorer(mae)

In [0]:
from fbprophet.plot import plot_plotly
import plotly.offline as py

In [45]:
weather_add_covid = pd.read_csv('wether_metro_covid.csv',encoding='CP949')
#weather_add_1
weather_add_covid['일시'] = pd.to_datetime(weather_add_covid['일시'], format='%Y-%m-%d', errors='raise')
weather_add_covid

,일시,평균기온(°C),일강수량(mm),평균 풍속(m/s),노선명,역명,승하차인원,휴일여부,confirmed
0,2019-01-01,-3.7,0.0,1.8,2호선,1,75273,1,0
1,2019-01-02,-3.5,0.0,1.5,2호선,1,233032,0,0
2,2019-01-03,-1.9,0.0,1.1,2호선,1,245143,0,0
3,2019-01-04,-0.5,0.0,1.3,2호선,1,264826,1,0
4,2019-01-05,-1.5,0.0,2.0,2호선,1,185844,1,0
...,...,...,...,...,...,...,...,...,...
4027,2020-05-14,20.9,0.0,0.4,5호선,8,19958,0,29
4028,2020-05-15,16.9,9.0,0.8,5호선,8,13201,1,27
4029,2020-05-16,17.0,1.5,0.6,5호선,8,11312,1,19
4030,2020-05-17,19.2,0.0,0.7,5호선,8,14847,1,13


In [46]:
#Facebook Prophet

df_set = weather_add_covid[weather_add_covid.역명 == 2]
df_prophet = pd.DataFrame({'ds' : df_set.일시, 'y' : df_set.승하차인원}).reset_index(drop=True)
df_prophet

,ds,y
0,2019-01-01,81143
1,2019-01-02,108547
2,2019-01-03,111173
3,2019-01-04,131287
4,2019-01-05,142740
...,...,...
499,2020-05-14,70149
500,2020-05-15,82554
501,2020-05-16,80678
502,2020-05-17,64649


In [55]:
for i in weather_add_covid.역명.unique():
  df_set = weather_add_covid[weather_add_covid.역명 == i]
  df_prophet = pd.DataFrame({'ds' : df_set.일시, 'y' : df_set.승하차인원}).reset_index(drop=True)
  prophet_model = Prophet() # 예측값에 상한과 하한을 두지 않을 것이기 때문에 'logistic' 등을 사용하지 않고 default인 'linear' 사용
  prophet_model.fit(df_prophet)
  future = prophet_model.make_future_dataframe(periods=7)
  forecast = prophet_model.predict(future)
  print(forecast['yhat'].tail(7))

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


504    141688.789285
505    140350.491928
506    142767.255568
507    158177.635880
508     97144.879299
509     28188.790401
510    135563.529049
Name: yhat, dtype: float64


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


504    36889.745260
505    38591.465157
506    41481.681504
507    61854.958309
508    71650.054631
509    43532.094384
510    34814.055450
Name: yhat, dtype: float64


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

504    51251.730394
505    52224.949359
506    53005.647669
507    61583.550898
508    55554.768420
509    31593.513509
510    48176.100768
Name: yhat, dtype: float64


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


504     8331.688070
505     9148.142012
506    10493.520088
507    18816.249547
508    24283.088422
509    10190.445038
510     6260.885263
Name: yhat, dtype: float64


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


504    12684.547555
505    12536.849201
506    12570.716528
507    15440.535439
508     9962.663154
509    -3311.207470
510     9705.815566
Name: yhat, dtype: float64
504    13588.884435
505    13664.260460
506    13259.391019
507    18212.936162
508    19594.313636
509    11558.696956
510    13638.990048
Name: yhat, dtype: float64


In [0]:
#평균적으로 금,토 승하차인원이 증가하는 추세이고, 신천지 코로나 사태이후에 급격하게 감소하는 추세라서 holiday에 반영해줌.
holiday = pd.DataFrame({
    'holiday': 'holiday',
    'ds': pd.concat([
        pd.Series(pd.date_range('2019-01-01', '2020-02-19', freq='W-FRI')),
        pd.Series(pd.date_range('2019-01-01', '2020-02-19', freq='W-SAT')),
        pd.Series(pd.date_range('2020-02-19', '2020-05-18', freq='D'))
        ])
})

In [0]:
holiday.sort_values(by="ds",ascending=True,inplace=True)

In [35]:
#additive 진폭이 거의 일정하여서
#additive -> 진폭일정 vs multiplicative -> 진폭 점점 증가하거나 감소할 때 설정
m = Prophet(
    seasonality_mode='additive',
    # holiday
    holidays=holiday,
    holidays_prior_scale=20
)   
print(m.history)

None


In [36]:
m.fit(df_prophet)
future = m.make_future_dataframe(periods=7)
forecast = m.predict(future)

fig = plot_plotly(m, forecast) 
py.iplot(fig)   

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [56]:
#holiday 파라미터 설정 후 영향을 20으로 설정한 후 mae값
#결과 이태원과 명동을 제외한 나머지 역은 holiday 설정안한 게 mae가 좋게 나옴.
#이태원과 명동만 holiday(금,토,코로나) 효과 있음
for i in weather_add_covid.역명.unique():
  df_set = weather_add_covid[weather_add_covid.역명 == i]
  size = int(len(df_set) * 0.8)
  train = df_set[:size]
  test = df_set[size:]
  # prophet은 ds와 y 두 개의 column으로만 진행
  df_prophet = pd.DataFrame({'ds' : train.일시, 'y' : train.승하차인원}).reset_index(drop=True) 
  m = Prophet(
    seasonality_mode='additive',
    # holiday
    holidays=holiday,
    holidays_prior_scale=20
    )
  result = m.fit(df_prophet)
  prophet_test = pd.DataFrame(test.일시).rename(columns={'일시' : 'ds'})
  prophet_pred_df = result.predict(prophet_test)
  prophet_pred = prophet_pred_df.yhat
  print("Mean value of 역명 {} :".format(i), df_set.승하차인원.mean())
  print("역명 {} MAE Score of test :".format(i), mae(test.승하차인원, prophet_pred))

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Mean value of 역명 1 : 188024.93055555556
역명 1 MAE Score of test : 72128.01545286369


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Mean value of 역명 2 : 108398.92857142857
역명 2 MAE Score of test : 51305.64104933861


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

Mean value of 역명 5 : 85433.3253968254
역명 5 MAE Score of test : 27415.138237308493


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Mean value of 역명 6 : 31289.059523809523
역명 6 MAE Score of test : 11753.829216580803


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Mean value of 역명 7 : 68663.45238095238
역명 7 MAE Score of test : 24988.79713199705
Mean value of 역명 8 : 22040.849206349205
역명 8 MAE Score of test : 8357.069884878727
